<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/banner_IA.png"  width="1000px" height="200px">

# **Taller 07:  Deep learning**

## **Outline**

1. [Ejercicio 1.](#eje1)
2. [Ejercicio 2.](#eje2)
3. [Ejercicio 3.](#eje3)
4. [Ejercicio 4.](#eje4)
5. [Ejercicio 5.](#eje5)



In [1]:
#@title **Execute this cell**
#@markdown Please include your student id
import sys
import inspect

group_id = "IA1-20252-G1" #@param {type:"string"}
assignment_id = group_id +'.taller_deep_learning'
student_id = "2204002" #@param {type:"string"}
"""
Put your student ID here

Example: student_id =  '2152145'
"""

"\nPut your student ID here\n\nExample: student_id =  '2152145'\n"

In [2]:
 #@title **Execute this cell**
#@markdown **UTILS**
#@markdown Please dont modify any line in this cell

import os
import json
import requests
from collections import namedtuple


Config = namedtuple('Config', ['server_name'])
config = Config(server_name='https://bivlabgrader.azurewebsites.net/api')


def check_solution_and_evaluate(assignment_id: str, student_func_str: str):

    # Set the endpoint and payload.
    payload = {
        'func_str': student_func_str,
        'assignment_id': assignment_id,
        'student_id': student_id
    }
    endpoint_url = config.server_name + '/CheckAndEvaluateSolution'
    # print(endpoint_url)

    # Make request to server with the data coming from the notebook.
    r = requests.post(endpoint_url, params=payload)
    pprint_json_response(r.json())
    return r


def pprint_json_response(response, indent=0):
    """Pretty print the response."""
    for key, value in response.items():
        print('\t' * indent + str(key.capitalize()))

        # If dictionary, do a recurrent call.
        if isinstance(value, dict):
            pprint_json_response(value, indent + 1)
        else:
            # Enumerate elements if list.
            if isinstance(value, list):
                if len(value) == 1:
                    print('\t' * (indent + 1) + str(value[0]))
                else:
                    for i, e in enumerate(value, start=1):
                        print('\t' * (indent + 1) + f'{i}. {e}')
            else:
                print('\t' * (indent + 1) + str(value))

In [3]:
#@title **Import libraries**

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(21)

import warnings
warnings.filterwarnings('ignore')

---
# **Ejercicio 1**  <a name="eje1"></a>
---

# Deep learning: classification

## Contexto:
Considere el dataset `penguins size` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/penguins_size.csv) el cual contiene datos sobre las dimensiones de una población de pinguinos.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


## Tu tarea
Implemente una función que **reciba** un dataset (`df`) y que:
- **Elimine** los valores nulos del dataset.
- Convierta las columnas categóricas `["species"], ["island"], ["sex"]` a numéricas.
- Considere a la columna `["sex"]` como el ground truth.
- Determine el número de clases del dataset (`nc`)
- Destine un 90% del dataset para el proceso de entrenamiento, usando también el parámetro `random_state=21`
- Entrene una red neuronal densa con:
  - Una capa densa con 256 unidades y activación relu
  - Una capa densa con 128 unidades y activación relu
  - Una capa densa con 64 unidades y activación relu
  - Una capa densa con activación sigmoide (asigne el número de unidades según lo visto para problemas de clasificación)
- Compile el modelo con parámetros: `optimizer=tf.keras.optimizers.SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy']`
- Entrene durante 10 `epochs`.
- **Devuelva** el número de clases (`nc`)
- **Devuelva** la pérdida del modelo
- **Devuelva** el accuracy el modelo

<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [4]:
def taller07_20252_p01(df):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    import tensorflow as tf
    from tensorflow import keras
    tf.random.set_seed(21)
    tf.keras.utils.set_random_seed(21)
    np.random.seed(21)

    # =======================
    # PREPROCESS COLUMNS
    # =======================

    # 1️⃣ Eliminar valores nulos
    df = df.dropna()

    # 2️⃣ Convertir columnas categóricas a numéricas
    df["species"], _ = pd.factorize(df["species"])
    df["island"], _ = pd.factorize(df["island"])
    df["sex"], _ = pd.factorize(df["sex"])

    # 3️⃣ Definir X (features) y y (ground truth)
    X = df.drop("sex", axis=1)
    y = df["sex"]

    # 4️⃣ Determinar el número de clases
    nc = len(y.unique())

    # =======================
    # DON'T DELETE
    # =======================
    X.columns = range(X.shape[1])
    X = X.to_numpy()
    y = y.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=21)
    # =======================

    # =======================
    # MODEL CREATION
    # =======================

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(nc, activation='sigmoid')
    ])

    # =======================
    # COMPILE AND TRAIN
    # =======================
    model.compile(
        optimizer=tf.keras.optimizers.SGD(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(X_train, y_train, epochs=10, verbose=0)

    # =======================
    # EVALUATE
    # =======================
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

    # =======================
    # RETURN
    # =======================
    return nc, loss, accuracy


In [5]:
#@title **check your answer**
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
taller07_20252_p01(df)

(3, 0.9815411567687988, 0.3529411852359772)

## ✅ Salidas esperadas

(3, 0.9815411567687988, 0.3529411852359772)


In [6]:
#@title **send your answer**
student_func_str = inspect.getsource(taller07_20252_p01)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all outputs are correct.


---
# **Ejercicio 2**  <a name="eje2"></a>
---

# Deep learning: classification

## Contexto
El dataset "80 Cereals" (https://www.kaggle.com/datasets/crawford/80-cereals) contiene información sobre distintas marcas de cereales, sus fabricantes, información nutricional, entre otros.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1DtVWAAUMDzVkh0NqDQT0i1JvTD8B4txv")
df.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


## Tu Tarea
Desarrolle una función que **reciba** un dataset (`df`) y que:
- Elimine los valores nulos del dataset.
- Elimine la columna `["name"]`
- Convierta las columnas categóricas `["mfr"]`, `["type"]` a numéricas.
- Considere como ground truth a la columna `["type"]` (tipo de cereal).
- Destine un 85% del dataset para el proceso de entrenamiento, usando también el parámetro `random_state=21`

- Determine el número de clases del dataset (`nc`)
- Entrene una red neuronal densa con:
  - Dos (2) capas densas con 128 unidades y activación relu
  - Una capa densa con 32 unidades y activación relu
  - Una capa densa con activación sigmoide (asigne el número de unidades según lo visto para problemas de clasificación)
- Compile el modelo con parámetros: `optimizer=tf.keras.optimizers.SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy']`
- Entrene durante 10 `epochs`.
- **Devuelva** el número de clases (`nc`)
- **Devuelva** la pérdida del modelo
- **Devuelva** el accuracy el modelo




<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [7]:
#@title **code student**

def taller07_20252_p02(df):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    import tensorflow as tf
    from tensorflow import keras

    # Fijar semillas para reproducibilidad
    tf.random.set_seed(21)
    tf.keras.utils.set_random_seed(21)
    np.random.seed(21)

    # ================================
    # Preprocess columns
    # ================================
    df = df.dropna()                     # Eliminar nulos
    if "name" in df.columns:
        df = df.drop(columns=["name"])   # Eliminar columna 'name'

    # Convertir columnas categóricas a numéricas
    df["mfr"], _ = pd.factorize(df["mfr"])
    df["type"], _ = pd.factorize(df["type"])

    # Definir X e y
    y = df["type"]
    X = df.drop(columns=["type"])

    # =====================================
    # NO MODIFICAR
    X.columns = range(X.shape[1])
    X = X.to_numpy()
    y = y.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.15, random_state=21
    )
    # =====================================

    # Determinar número de clases
    nc = len(np.unique(y))

    # Construir modelo de red neuronal
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(nc, activation='sigmoid')
    ])

    # Compilar modelo
    model.compile(
        optimizer=tf.keras.optimizers.SGD(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Entrenar modelo
    history = model.fit(X_train, y_train, epochs=10, verbose=0)

    # Evaluar modelo
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

    # Retornar número de clases, pérdida y exactitud
    return nc, loss, accuracy

In [8]:
#@title **check your answer**
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1DtVWAAUMDzVkh0NqDQT0i1JvTD8B4txv")
taller07_20252_p02(df)

(2, 0.529904842376709, 0.9166666865348816)

## ✅ Salidas esperadas

(2, 0.529904842376709, 0.9166666865348816)


In [9]:
#@title **send your answer**
student_func_str = inspect.getsource(taller07_20252_p02)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all outputs are correct.


---
# **Ejercicio 3**  <a name="eje3"></a>
---

# Deep learning: classification

## Contexto
El dataset `supermarket sales` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/supermarket_sales.csv) contiene información relacionada con productos, costos, ganancias y otros aspectos de un supermercado.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


## Tu tarea

Implementar una función que **reciba** como parámetro el dataset (`df`), y que:
- Elimine las columnas `["Invoice ID", "Date", "Time"]`
- Convierta las columnas categóricas `["Branch"], ["City"], ["Customer type"], ["Gender"], ["Product line"], ["Payment"]` a numéricas.
- El ground truth será la columna `["Branch"]`
- Utilice un 80% del dataset para entrenamiento, con parámetros `test_size=0.2, random_state=21`)
- Determine el número de clases del dataset (`nc`)
- Entrene una red neuronal densa con:
  - Dos (3) capas densas con 256 unidades y activación relu
  - Una capa densa con activación sigmoide (asigne el número de unidades según lo visto para problemas de clasificación)
- Compile el modelo con parámetros: `optimizer=tf.keras.optimizers.SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy']`
- Entrene durante 10 `epochs`.
- **Devuelva** el número de clases (`nc`)
- **Devuelva** la pérdida del modelo
- **Devuelva** el accuracy el modelo






<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [10]:
#@title **code student**

def taller07_20252_p03(df):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    import tensorflow as tf
    from tensorflow import keras

    # Fijar semillas para reproducibilidad
    tf.random.set_seed(21)
    tf.keras.utils.set_random_seed(21)
    np.random.seed(21)

    # ================================
    # Preprocess columns
    # ================================
    # Eliminar columnas innecesarias
    df = df.drop(columns=["Invoice ID", "Date", "Time"], errors='ignore')

    # Convertir columnas categóricas a numéricas
    for col in ["Branch", "City", "Customer type", "Gender", "Product line", "Payment"]:
        df[col], _ = pd.factorize(df[col])

    # Definir X e y (ground truth = Branch)
    y = df["Branch"]
    X = df.drop(columns=["Branch"])

    # =====================================
    # DON'T DELETE! ************************
    X.columns = range(X.shape[1])
    X = X.to_numpy()
    y = y.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=21
    )
    # =====================================

    # Determinar número de clases
    nc = len(np.unique(y))

    # Definir el modelo
    model = keras.Sequential([
        keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(nc, activation='sigmoid')
    ])

    # Compilar el modelo
    model.compile(
        optimizer=tf.keras.optimizers.SGD(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # Entrenar el modelo
    history = model.fit(X_train, y_train, epochs=10, verbose=0)

    # Evaluar modelo
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

    # Retornar resultados
    return nc, loss, accuracy

In [11]:
#@title **check your answer**
import numpy as np
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
taller07_20252_p03(df)

(3, 1.0972102880477905, 0.3100000023841858)

## ✅ Salidas esperadas

(3, 1.0972102880477905, 0.3100000023841858)


In [12]:
#@title **send your answer**
student_func_str = inspect.getsource(taller07_20252_p03)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all outputs are correct.


---
# **Ejercicio 4**  <a name="eje4"></a>
---

# Deep learning: regression

## Contexto
El dataset `supermarket sales` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/supermarket_sales.csv) contiene información relacionada con productos, costos, ganancias y otros aspectos de un supermercado.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


## Tu tarea

Implementar una función que **reciba** como parámetro el dataset (`df`), y que:
- Elimine las columnas `["Invoice ID", "Date", "Time", "gross margin percentage"]`
- Convierta las columnas categóricas `["Branch"], ["City"], ["Customer type"], ["Gender"], ["Product line"], ["Payment"]` a numéricas.
- El ground truth será la columna `["Unit price"]`
- Utilice un 90% del dataset para entrenamiento, con parámetros `test_size=0.1, random_state=21`)

- Entrene una red neuronal densa con:
  - Una capa densa con 128 unidades y activación relu
  - Dos (2) capas densas con 256 unidades y activación relu
  - Una capa densa (asigne el número de unidades según lo visto para problemas de <font color="red">regresión</font>)
- Compile el modelo con parámetros: `optimizer=tf.keras.optimizers.SGD(),
              loss='mae',
              metrics=['mae','mse']`
- Entrene durante 10 `epochs`.
- **Devuelva** la pérdida del modelo



<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [19]:
#@title **code student**

def taller07_20252_p04(df):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    import tensorflow as tf
    from tensorflow import keras

    tf.random.set_seed(21)
    tf.keras.utils.set_random_seed(21)
    np.random.seed(21)

    # ============================
    # preprocess columns
    # ============================

    # 1️⃣ Eliminar columnas innecesarias
    df = df.drop(columns=["Invoice ID", "Date", "Time", "gross margin percentage"], errors='ignore')

    # 2️⃣ Convertir columnas categóricas a numéricas
    for col in ["Branch", "City", "Customer type", "Gender", "Product line", "Payment"]:
        df[col], _ = pd.factorize(df[col])

    # 3️⃣ Definir X (features) y y (target)
    y = df["Unit price"]
    X = df.drop(columns=["Unit price"])

    # =========================================
    # DON'T DELETE! ****************************
    X.columns = range(X.shape[1])
    X = X.to_numpy()
    y = y.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=21
    )
    # =========================================

    # 4️⃣ Definir red neuronal densa
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(1)  # salida para regresión
    ])

    # 5️⃣ Compilar modelo
    model.compile(
        optimizer=tf.keras.optimizers.SGD(),
        loss='mae',
        metrics=['mae', 'mse']
    )

    # 6️⃣ Entrenar modelo
    model.fit(X_train, y_train, epochs=10, verbose=0)

    # 7️⃣ Evaluar modelo
    loss, mae, mse = model.evaluate(X_test, y_test, verbose=0)

    # 8️⃣ Retornar salidas esperadas
    return loss, mae, mse

In [20]:
#@title **check your answer**
import numpy as np
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
taller07_20252_p04(df)

(22.8461971282959, 22.8461971282959, 697.533447265625)

## ✅ Salidas esperadas

[22.8461971282959, 22.8461971282959, 697.533447265625]


In [21]:
#@title **send your answer**
student_func_str = inspect.getsource(taller07_20252_p04)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all outputs are correct.


---
# **Ejercicio 5**  <a name="eje5"></a>
---

# Deep learning: regression

## Contexto:
Considere el dataset `penguins size` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/penguins_size.csv) el cual contiene datos sobre las dimensiones de una población de pinguinos.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


## Tu tarea
Implemente una función que **reciba** un dataset (`df`) y que:
- **Elimine** los valores nulos del dataset.
- Convierta las columnas categóricas `["species"], ["island"], ["sex"]` a numéricas.
- Considere a la columna `["flipper_length_mm"]` como el ground truth.

- Utilice un 90% del dataset para entrenamiento, con parámetros `test_size=0.1, random_state=21`)

- Entrene una red neuronal densa con:
  - Dos (2) capas densas con 128 unidades y activación relu
  - Dos (2) capas densas con 256 unidades y activación relu
  - Una capa densa (asigne el número de unidades según lo visto para problemas de <font color="red">regresión</font>)
- Compile el modelo con parámetros: `optimizer=tf.keras.optimizers.SGD(),
              loss='mae',
              metrics=['mae','mse']`
- Entrene durante 10 `epochs`.
- **Devuelva** la pérdida del modelo



<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [22]:
#@title **code student**

def taller07_20252_p05(df):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    import tensorflow as tf
    from tensorflow import keras
    tf.random.set_seed(21)
    tf.keras.utils.set_random_seed(21)
    np.random.seed(21)

    # ===========================================
    # preprocess columns
    # ===========================================
    # 1️⃣ Eliminar valores nulos
    df = df.dropna()

    # 2️⃣ Convertir columnas categóricas a numéricas
    for col in ["species", "island", "sex"]:
        df[col] = pd.factorize(df[col])[0]

    # 3️⃣ Definir la variable objetivo
    y = df["flipper_length_mm"]
    X = df.drop(columns=["flipper_length_mm"])

    # ===========================================
    # DON'T DELETE! *****************************
    X.columns = range(X.shape[1])
    X = X.to_numpy()
    y = y.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=21
    )
    # ===========================================

    # 4️⃣ Crear el modelo
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(1)  # capa de salida para regresión
    ])

    # 5️⃣ Compilar el modelo
    model.compile(
        optimizer=tf.keras.optimizers.SGD(),
        loss='mae',
        metrics=['mae', 'mse']
    )

    # 6️⃣ Entrenar el modelo
    model.fit(X_train, y_train, epochs=10, verbose=0)

    # 7️⃣ Evaluar el modelo
    loss, mae, mse = model.evaluate(X_test, y_test, verbose=0)

    # 8️⃣ Devolver resultados
    return [loss, mae, mse]

In [23]:
#@title **check your answer**
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
taller07_20252_p05(df)

[200.05999755859375, 200.05999755859375, 40196.3515625]

## ✅ Salidas esperadas

[200.05999755859375, 200.05999755859375, 40196.3515625]


In [24]:
#@title **send your answer**
student_func_str = inspect.getsource(taller07_20252_p05)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all outputs are correct.


---
<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/bannerThanks.jpg" alt="Drawing" style="width:700px;"/>